# DEEP SPEECH1 구현 - pytorch

CTC loss 를 이용한 deep speech 구현이다. <br>
구현 모델은 [Deep speech 이론 by YBigTa](https://github.com/YBIGTA/Deep_learning/blob/master/RNN/deep%20speech/%EC%84%A4%EB%AA%85/Deep%20speech_%EC%83%81%ED%97%8C.pdf) 에서 확인 가능하다. <br>

본 코드는 [deep speech2 implementation](https://github.com/SeanNaren/deepspeech.pytorch/)을 상당부분 참고한다.

### 구현 stack
OS : ubuntu 16.04 <br>
conda : 4.2.9 <br>


## 설치

오디오 I/O 를 위한 pytorch audio를 설치한다. <br>
```
sudo apt-get install sox libsox-dev libsox-fmt-all
git clone https://github.com/pytorch/audio.git
cd audio
python setup.py install
```

Requirments를 설치한다. <br>
```
sudo pip install python-levenshtein torch visdom wget librosa
```

## 0. 데이터셋 다운로드

데이터셋의 경우 카네기 멜론 대학교에서 제공한 free dataset인 **AN4** 를 사용한다.

```python
import argparse
import os
import io
import shutil
import tarfile
import wget

import subprocess

def _order_files(file_paths):
    print("Sorting files by length...")

    def func(element):
        output = subprocess.check_output(
            ['soxi -D \"%s\"' % element.strip()],
            shell=True
        )
        return float(output)

    file_paths.sort(key=func)
    
def create_manifest(data_path, tag, ordered=True):
    manifest_path = '%s_manifest.csv' % tag
    file_paths = []
    wav_files = [os.path.join(dirpath, f)
                 for dirpath, dirnames, files in os.walk(data_path)
                 for f in fnmatch.filter(files, '*.wav')]
    size = len(wav_files)
    counter = 0
    for file_path in wav_files:
        file_paths.append(file_path.strip())
        counter += 1
    print('\n')
    if ordered:
        _order_files(file_paths)
    counter = 0
    with io.FileIO(manifest_path, "w") as file:
        for wav_path in file_paths:
            transcript_path = wav_path.replace('/wav/', '/txt/').replace('.wav', '.txt')
            sample = os.path.abspath(wav_path) + ',' + os.path.abspath(transcript_path) + '\n'
            file.write(sample.encode('utf-8'))
            counter += 1
    print('\n')

# command line에서 동작하는 것들을 더 쉽게 만들어주는 parser를 이용하여 데이터를 다운받는다.
parser = argparse.ArgumentParser(description='Processes and downloads an4.') 
parser.add_argument('--target_dir', default='an4_dataset/', help='Path to save dataset')
parser.add_argument('--sample_rate', default=16000, type=int, help='Sample rate')
args = parser.parse_args()


def _format_data(root_path, data_tag, name, wav_folder):
    data_path = args.target_dir + data_tag + '/' + name + '/'
    new_transcript_path = data_path + '/txt/'
    new_wav_path = data_path + '/wav/'

    os.makedirs(new_transcript_path)
    os.makedirs(new_wav_path)

    wav_path = root_path + 'wav/'
    file_ids = root_path + 'etc/an4_%s.fileids' % data_tag
    transcripts = root_path + 'etc/an4_%s.transcription' % data_tag
    train_path = wav_path + wav_folder

    _convert_audio_to_wav(train_path)
    _format_files(file_ids, new_transcript_path, new_wav_path, transcripts, wav_path)


def _convert_audio_to_wav(train_path):
    with os.popen('find %s -type f -name "*.raw"' % train_path) as pipe:
        for line in pipe:
            raw_path = line.strip()
            new_path = line.replace('.raw', '.wav').strip()
            cmd = 'sox -t raw -r %d -b 16 -e signed-integer -B -c 1 \"%s\" \"%s\"' % (
                args.sample_rate, raw_path, new_path)
            os.system(cmd)


def _format_files(file_ids, new_transcript_path, new_wav_path, transcripts, wav_path):
    with open(file_ids, 'r') as f:
        with open(transcripts, 'r') as t:
            paths = f.readlines()
            transcripts = t.readlines()
            for x in range(len(paths)):
                path = wav_path + paths[x].strip() + '.wav'
                filename = path.split('/')[-1]
                extracted_transcript = _process_transcript(transcripts, x)
                current_path = os.path.abspath(path)
                new_path = new_wav_path + filename
                text_path = new_transcript_path + filename.replace('.wav', '.txt')
                with io.FileIO(text_path, "w") as file:
                    file.write(extracted_transcript.encode('utf-8'))
                os.rename(current_path, new_path)


def _process_transcript(transcripts, x):
    extracted_transcript = transcripts[x].split('(')[0].strip("<s>").split('<')[0].strip().upper()
    return extracted_transcript


def main():
    root_path = 'an4/'
    name = 'an4'
    wget.download('http://www.speech.cs.cmu.edu/databases/an4/an4_raw.bigendian.tar.gz')
    tar = tarfile.open('an4_raw.bigendian.tar.gz')
    tar.extractall()
    os.makedirs(args.target_dir)
    _format_data(root_path, 'train', name, 'an4_clstk')
    _format_data(root_path, 'test', name, 'an4test_clstk')
    shutil.rmtree(root_path)
    os.remove('an4_raw.bigendian.tar.gz')
    train_path = args.target_dir + '/train/'
    test_path = args.target_dir + '/test/'
    print ('\n', 'Creating manifests...')
    create_manifest(train_path, 'an4_train')
    create_manifest(test_path, 'an4_val')


if __name__ == '__main__':
    main()
```

## 1.1 Making audio spectogram using fourier transformation

## Dataset label csv 파일 제작

### training, testing을 위한 data는 다음과 같은 형태로 폴더 내에 저장이 되어 있다. 

#### training 

*data/data_path_to_train/input* <br>
*data/data_path_to_train/label* <br>

#### testing

*data/data_path_to_test/input* <br>
*data/data_path_to_test/label* <br>

따라서 본 데이터를 load 하기 위하여 training 시, testing 시 각각 불러올 파일들의 path를 하나의 csv파일(manifest라 부른다.) 로 저장하면 부르기에 용이하다. <br>

위의 코드 중 create_manifest 함수가 본 파일을 csv화 해주는 함수이다. <br>

따라서 코드를 돌린 장소에 csv 파일을 참조한다.


## Dataset과 Dataloader 구현

pytorch의 dataset과 dataloader의 경우는 그 클래스가 정해져 있다. <br>

따라서 data loader와 dataset의 경우에는 사용자 정의 클래스를 만들기 위해서 **torch.utils.data**의 **Dataset**과 **DataLoader**를 상속받아야 한다. <br>

이를 통해 사용자 정의 dataset, dataloader를 구현할 것이며, <br>

본 dataset에는 audio spectogram 형태의 dataset이여야 하므로 *wav* 파일을 *spectogram* 형태로 바꾸는 함수를 집어넣고 구현한다., <br>

추가적인 dataset, dataloader 구현 튜토리얼은 [pytorch 공인 튜토리얼](http://pytorch.org/tutorials/beginner/data_loading_tutorial.html) 을 참고한다.

In [61]:
# 필요 class load
import librosa
import numpy as np
import scipy.signal
import torch
import torchaudio
import copy
from torch.utils.data import DataLoader
from torch.utils.data import Dataset



In [62]:
"""
wav 파일을 load하는 함수.
torchaudio를 사용한다. 

Input : 
    wav 파일 path. 자료형 : str
Output :
    오디오 파일의 numpy 형태. 자료형 : np
"""
def load_audio(path):
    sound, _ = torchaudio.load(path)
    sound = sound.numpy()
    if len(sound.shape) > 1:
        if sound.shape[1] == 1:
            sound = sound.squeeze()
        else:
            sound = sound.mean(axis=1)  # multiple channels, average
    return sound

In [65]:
"""
Fourier transformation parser.
spectogramparser는 spectogramDataset에 상속되며, 모든 wav 파일을 spectogram으로 파싱하는 클래스이다.

Param : 
    audio_conf - 오디오 특성의 딕셔너리. spectogram화를 위한 window 방식, window size, stride, 음성의 rate 가 들어있어야 한다. 자료형 : dict
"""
class SpectogramParser(object):
    # 초기화 함수. 
    def __init__(self, audio_conf):
        super(SpectogramParser, self).__init__()
        self.window_stride = audio_conf['window_stride']
        self.window_size = audio_conf['window_size']
        self.sample_rate = audio_conf['sample_rate']
        self.window = audio_conf['window']
    
    """
    parsing 함수. 
    Input : 
        audio_path. 자료형 : str or list
    output : 
        해당 path의 spectogram, 자료형 : FloatTensor (MHz + 1, len)
    """
    def parse_audio(self, audio_path):
        # 하나의 path가 string형태로 들어온다면
        if(type(audio_path)==str):
            y = load_audio(audio_path)
            n_fft = int(self.sample_rate * self.window_size)
            win_length = n_fft
            hop_length = int(self.sample_rate * self.window_stride)
            # STFT
            D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length,
                             win_length=win_length, window=self.window)
            spect, phase = librosa.magphase(D)
            # S = log(S+1)
            spect = np.log1p(spect)
            spect = torch.FloatTensor(spect)
            return spect
        # 굳이 필요 없음이 밝혀짐.
        # 여러개의 path가 list형태로 들어온다면 tmp에 저장
#        elif(type(audio_path)==list):
#            tmp = ""
#            for path in audio_path:
#                y = load_audio(path)
#                n_fft = int(self.sample_rate * self.window_size)
#                win_length = n_fft
#                hop_length = int(self.sample_rate * self.window_stride)
                # STFT
#                D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length,
#                                 win_length=win_length, window=self.window)
#                spect, phase = librosa.magphase(D)
                # S = log(S+1)
#                spect = np.log1p(spect)
#                shpe = spect.shape
#                spect = spect.reshape((1,)+shpe)
#                if(tmp == ""):
#                    tmp = copy.deepcopy(spect)
#                else:
#                    tmp = np.append(tmp,spect,axis=0)
#                
#            spect = torch.FloatTensor(tmp)
#            return spect               
        else:
            print("wrong type of audiopath")
            return -1

#### *해당 클래스 example*

In [68]:
# sample.wav 파일로 테스트. 본 파일은 an4 training set의 첫 번째 data이다.
audio_conf = {}
audio_conf["sample_rate"] = torchaudio.load("./sample.wav")[1] #torchaudio의 2번째 return value는 해당 wav 파일의 rate이다.
audio_conf["window_size"] = 0.02
audio_conf["window_stride"] = 0.01
audio_conf["window"] = scipy.signal.hamming

#class 초기화
parser =SpectogramParser(audio_conf)

#parsing된 데이터
parser.parse_audio("./sample.wav")


 17.1550  18.0746  17.1650  ...   16.8284  15.2061  16.6281
 15.7563  17.7952  16.6147  ...   17.0611  17.5940  16.2806
 15.8626  17.1986  16.3381  ...   16.0992  17.9769  14.8790
           ...               ⋱              ...            
 16.2552  14.8504  16.8628  ...   15.0124  14.9490  15.1515
 16.0438  15.2556  15.3839  ...   15.0961  14.9769  15.6644
 15.8748  14.8320  15.4217  ...   14.0025  14.5311  15.2816
[torch.FloatTensor of size 161x151]

In [69]:
# sample.wav 파일로 테스트. 본 파일은 an4 training set의 첫 번째 data이다.
audio_conf = {}
audio_conf["sample_rate"] = torchaudio.load("./sample2.wav")[1] #torchaudio의 2번째 return value는 해당 wav 파일의 rate이다.
audio_conf["window_size"] = 0.02
audio_conf["window_stride"] = 0.01
audio_conf["window"] = scipy.signal.hamming

#class 초기화
parser =SpectogramParser(audio_conf)

#parsing된 데이터
parser.parse_audio("./sample2.wav")


  9.9548  12.5342  12.5185  ...   21.6904  21.7963  21.1424
  9.9540  12.6197  13.0785  ...   23.0774  21.1615  22.5326
  9.9516  12.7327  12.4498  ...   23.8243  23.5357  22.5847
           ...               ⋱              ...            
  1.5135  11.5833  11.7393  ...   15.0627  16.1645  16.5222
  2.5937  11.3482  11.6363  ...   15.1313  16.1363  16.5455
  2.7930  11.0813  12.4409  ...   15.2082  16.0953  16.4942
[torch.FloatTensor of size 442x11627]

## Dataset 구현

In [70]:
"""
pytorch의 dataset 구현. torch의 Dataset, SpectogramParser를 상속받는다.
또한 *parse_transcript* 함수를 보면, 해당 text 가 index로 리턴이 되어야 한다. 이 또한 Data loader에 구현이 되는 것이다.

Input :
    audio_conf - 부모 클래스 spectogramparser 초기화를 위한 인자. 자료형 : dict
    manifest_filepath - wav, input 으로 나뉘어져 있는 데이터의 경로가 저장되어있는 csv 파일. 자료형 : str
    labels - 우리가 관측하고 싶은 character들. 자료형 : str
"""
class SpectogramDataset(Dataset, SpectogramParser):
    #초기화 함수. label을 index로 변환하기 위한 딕셔너리 형태인 labels_map을 만들어야 한다.
    def __init__(self, audio_conf, manifest_filepath, labels):
        with open(manifest_filepath) as f:
            ids = f.readlines()
        ids = [x.strip().split(',') for x in ids]
        self.ids = ids
        self.size = len(ids)
        self.labels_map = dict([(labels[i], i) for i in range(len(labels))])
        super(SpectogramDataset, self).__init__(audio_conf)

    def __getitem__(self, index):
        sample = self.ids[index]
        audio_path, transcript_path = sample[0], sample[1]
        spect = self.parse_audio(audio_path)
        transcript = self.parse_transcript(transcript_path)
        return spect, transcript

    def parse_transcript(self, transcript_path):
        with open(transcript_path, 'r') as transcript_file:
            transcript = transcript_file.read().replace('\n', '')
        transcript = list(filter(None, [self.labels_map.get(x) for x in list(transcript)]))
        return transcript

    def __len__(self):
        return self.size

#### *해당 클래스 example*

In [137]:
# sample.wav 파일로 테스트. 본 파일은 an4 training set의 첫 번째 data이다.
audio_conf = {}
audio_conf["sample_rate"] = torchaudio.load("./sample.wav")[1] #torchaudio의 2번째 return value는 해당 wav 파일의 rate이다.
audio_conf["window_size"] = 0.02
audio_conf["window_stride"] = 0.01
audio_conf["window"] = scipy.signal.hamming

#초기화
dataset = SpectogramDataset(audio_conf,"an4_train_manifest.csv", "_'ABCDEFGHIJKLMNOPQRSTUVWXYZ ")

In [73]:
# 지금 slice 가 먹지를 않는다. parse audio 가 list를 인식하지 못함. 근데 굳이 slice가 먹혀야 하는가?
dataset[1]

(
  19.2830  19.3306  19.0667  ...   19.8727  19.3608  18.9193
  18.7680  18.6099  18.4839  ...   19.1430  18.4066  17.5571
  18.0558  17.3596  17.7159  ...   17.1681  17.3474  16.8489
            ...               ⋱              ...            
  15.5034  15.6864  16.3356  ...   14.3198  15.6247  15.4818
  12.8209  16.2215  16.3916  ...   15.3400  15.7537  14.3083
  16.0653  10.6976  15.2672  ...   14.5831  12.9391  15.2190
 [torch.FloatTensor of size 161x71], [9, 6, 13, 17])

## DataLoader 구현

In [74]:
"""
collate fn 함수는 dataloader에서 minibatch를 만들어주는 default 함수이다.
음성인식에서 minibatch를 하는 기본적인 기조는, 일단 매 음성데이터의 사이즈가 다르므로, 가장 큰 음성인식 파일에 다른 파일들의 크기를 맞추는 것이다.
input :
    batch - List of dataset. dataset에서 샘플링한 것의 리스트이다.
output :
    inputs - 4 차원의 input. N x C X H * W. 따라서 본 deep speech에서는 C는 항상 1이다.
    targets - 1 차원의 label을 가진 IntTensor. 4번째 hidden layer를 지나면서 모든 minibatch는 concatenate된다. 따라서 1차원이다.
    input_percentages - 1차원의 각 sample의 가장 큰 sample에 비한 상대적인 크기. 즉, 얼마만큼의 0이 삽입되었는가를 알려준다. CTC loss를 계산하는데 사용.
    target_sizes - 각 글자 수.
"""
def _collate_fn(batch):
    def func(p):
        return p[0].size(1) # 길이.

    longest_sample = max(batch, key=func)[0]
    freq_size = longest_sample.size(0)
    minibatch_size = len(batch)
    max_seqlength = longest_sample.size(1) #가장 긴 sample의 길이
    inputs = torch.zeros(minibatch_size, 1, freq_size, max_seqlength) # N X C X H X W
    input_percentages = torch.FloatTensor(minibatch_size) 
    target_sizes = torch.IntTensor(minibatch_size)
    targets = []
    for x in range(minibatch_size):
        sample = batch[x]
        tensor = sample[0]
        target = sample[1]
        seq_length = tensor.size(1)
        inputs[x][0].narrow(1, 0, seq_length).copy_(tensor)
        input_percentages[x] = seq_length / float(max_seqlength)
        target_sizes[x] = len(target)
        targets.extend(target)
    targets = torch.IntTensor(targets)
    return inputs, targets, input_percentages, target_sizes

In [75]:
"""
DataLoader.
enumerate로 돌아야 하며, i, (collate_fn output) 으로 iteration이 돈다.
초기화 input: 
    dataset : spectogramdataset이 될 것
    minibatch size : default might be 20
    worker : loading을 위한 process의 갯수.
"""
class AudioDataLoader(DataLoader):
    def __init__(self, *args, **kwargs):
        super(AudioDataLoader, self).__init__(*args, **kwargs)
        self.collate_fn = _collate_fn

In [138]:
loader = AudioDataLoader(dataset,2,4)

## 2. modeling

모델의 자세한 프로세스는 다음의 그림을 참고한다 <br>
[딥 스피치 모델](https://github.com/YBIGTA/Deep_learning/blob/master/RNN/deep_speech/implementation/%EB%AA%A8%EB%8D%B8_%EB%94%A5%EC%8A%A4%ED%94%BC%EC%B9%98.pdf)

In [3]:
import math
from collections import OrderedDict
import torch
import torch.nn as nn
import torch.nn.functional as F



In [1]:
"""
Collapses input of dim T*N*H to (T*N)*H, and applies to a module.
Allows handling of variable sequence lengths and minibatch sizes.
:param module: Module to apply input to.
"""
"""
총 두 번 사용된다. CNN의 output이 RNN으로 들어갈 때. RNN의 output이 FC로 들어갈 때. 
그 필요성은 위의 딥 스피치 모델 그림에서 데이터가 transpose됨을 보면 알 수 있다.
"""
class SequenceWise(nn.Module):
    def __init__(self, module):
        super(SequenceWise, self).__init__()
        self.module = module

    def forward(self, x):
        t, n = x.size(0), x.size(1)
        x = x.view(t * n, -1)
        x = self.module(x)
        x = x.view(t, n, -1)
        return x

"""
이건 굉장히 typical 한 minibatch softmax.
각 data의 softmax를 torch.stack을 해야한다는 부분이 했심.
QQQQQQQQQQQ 왜 if not self.training인가. 
"""
class InferenceBatchSoftmax(nn.Module):
    def forward(self, input_):
        if not self.training:
            batch_size = input_.size()[0]
            return torch.stack([F.log_softmax(input_[i]) for i in range(batch_size)], 0)
        else:
            return input_
        
#class BatchSoftmax(nn.Module):
#    def forward(self, input_):
#        return torch.stack([F.log_softmax(input_[i] for i in range(batch_size))], 0)
        
"""
batchnormalization + RNN 구현.
굳이 이런식으로 붙이는 이유는 sequencewise 때문에. 
input : 
    input_size - T * N * H 에서 H 의 값. 여기서 T 는 time, N 은 mini batch 갯수, H는 P * C(convolution layers)
    hidden_size - output의 값. 
                  여기서 output은 input과 같지 않냐! 고 생각하는데, 단순히 hi = W1 * xi + W2 * xi 라고 했을 때 matrix의 행의 크기. 
                  즉, T * N * H에서 H가 얼마나 압축될 것이냐의 의미.
"""
class BatchRNN(nn.Module):
    def __init__(self, input_size, hidden_size, rnn_type=nn.RNN, bidirectional=True, batch_norm=True):
        super(BatchRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional
        self.batch_norm = SequenceWise(nn.BatchNorm1d(input_size)) if batch_norm else None
        self.rnn = rnn_type(input_size=input_size, hidden_size=hidden_size,
                            bidirectional=bidirectional, bias=False)
        self.num_directions = 2 if bidirectional else 1 # why do I need this?

    def forward(self, x):
        if self.batch_norm is not None:
            x = self.batch_norm(x)
        x, _ = self.rnn(x)
        if self.bidirectional:
            x = x.view(x.size(0), x.size(1), 2, -1).sum(2).view(x.size(0), x.size(1), -1)  # (TxNxH*2) -> (TxNxH) by sum
            return x

NameError: name 'nn' is not defined

In [ ]:

"""
deep speech model. 
Input : 
    없어도 된다!
"""
class DeepSpeech(nn.Module):
    def __init__(self, rnn_type=nn.RNN, labels="_'ABCDEFGHIJKLMNOPQRSTUVWXYZ ", rnn_hidden_size=768, audio_conf=None,
                 bidirectional=True):
        super(DeepSpeech, self).__init__()

        # model metadata needed for serialization/deserialization
        if audio_conf is None:
            audio_conf = {}
        self._hidden_size = rnn_hidden_size
        self._hidden_layers = nb_layers
        self._rnn_type = rnn_type
        self._audio_conf = audio_conf or {}
        self._labels = labels

        sample_rate = self._audio_conf.get("sample_rate", 16000)
        window_size = self._audio_conf.get("window_size", 0.02)
        num_classes = len(self._labels)

        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(41, 11), stride=(2, 2)),
            nn.BatchNorm2d(32),
            nn.Hardtanh(0, 20, inplace=True),
            nn.Conv2d(32, 32, kernel_size=(21, 11), stride=(2, 1)),
            nn.BatchNorm2d(32),
            nn.Hardtanh(0, 20, inplace=True)
        )
        # Based on above convolutions and spectrogram size using conv formula (W - F + 2P)/ S+1
        rnn_input_size = int(math.floor((sample_rate * window_size) / 2) + 1)
        rnn_input_size = int(math.floor(rnn_input_size - 41) / 2 + 1)
        rnn_input_size = int(math.floor(rnn_input_size - 21) / 2 + 1)
        rnn_input_size *= 32

        rnns = []
        self.rnn = BatchRNN(input_size=rnn_input_size, hidden_size=rnn_hidden_size, rnn_type=rnn_type,
                       bidirectional=bidirectional, batch_norm=True)

        fully_connected = nn.Sequential(
            nn.BatchNorm1d(rnn_hidden_size),
            nn.Linear(rnn_hidden_size, num_classes, bias=False)
        )
        self.fc = nn.Sequential(
            SequenceWise(fully_connected),
        )
        self.softmax = InferenceBatchSoftmax()

    def forward(self, x):
        x = self.coLnv(x)

        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # Collapse feature dimension
        x = x.transpose(1, 2).transpose(0, 1).contiguous()  # TxNxH

        x = self.rnn(x)

        x = self.fc(x)
        x = x.transpose(0, 1)
        x = self.softmax(x)
        return x

## Loss function 정의.

CTC loss를 정의하여야 한다. <br>
[3 페이지 참조](https://github.com/YBIGTA/Deep_learning/blob/master/RNN/deep_speech/%EC%84%A4%EB%AA%85/Deep%20speech_%EC%83%81%ED%97%8C.pdf)

현재 CTC loss는 pytorch에서 제공을 하고있지 않다. <br>

따라서 [torch wrapper file](https://github.com/baidu-research/warp-ctc/blob/master/torch_binding/binding.cpp)을 [pytorch 로 transpile](https://github.com/pytorch/extension-ffi) 해야한다. <br>